In [1]:
using LogDensityProblems: LogDensityProblems;
using Distributions
using DelimitedFiles
using Random: Random
using DynamicPPL

data = readdlm("data.txt")
t, y, yerr = data[:, 1], data[:, 2], data[:, 3]


([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5  …  10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0], [-7.847359573362729, -1.3315947853190484, -0.5803510819209906, 1.1102764392598465, 9.988712842023517, 19.064552472027945, 22.360242601879634, 33.29037569667121, 44.30268624603188, 61.37029298584822  …  329.46633376403497, 367.3951090879816, 401.4586394961345, 429.0802630665355, 468.61496107560146, 507.32002150234865, 545.8993822489858, 589.8509862829196, 640.5342047989809, 675.5419963803967], [2.3730589643393882, 1.3446357078635467, 0.21190807702627767, 2.4126456988021374, 1.4574964737150893, 4.5564066433504316, 2.007971577637596, 1.5027663187179967, 2.6671045028916103, 1.8335143799268723  …  2.4187475006856918, 3.7358953477686847, 3.7144608329447912, 4.750626684174312, 0.10958338163060743, 4.894885896061309, 0.732250812640492, 2.748137898114247, 4.061190017100702, 0.033932572914373016])

In [2]:
using Turing

In [10]:
# Let's define some type that represents the model.
struct RegressionProblem{Ty <: AbstractVector}
	"mean of the isotropic Gaussian"
	y::Ty
	x::Ty
	# α::A
	# β::A
	# γ::A
end
LogDensityProblems.dimension(model::RegressionProblem) = 3

function LogDensityProblems.logdensity(model::RegressionProblem, θ::AbstractVector{<:Real})
	α, β, γ = θ
	lp = logpdf(Normal(2, 1), α)
	lp += logpdf(Normal(0, 1), β)
	lp += logpdf(Normal(-2, 1), γ)
	ll = logpdf(MvNormal(fun(model.x, α, β, γ), 1), model.y)
	return ll + lp
end


fun(x,α,β,γ) = @. α * x^2 + β * x + γ
x = 0.0:0.5:15.0

LogDensityProblems.capabilities(model::RegressionProblem) = LogDensityProblems.LogDensityOrder{0}()

In [11]:
MYmodel = RegressionProblem(y,t)

RegressionProblem{Vector{Float64}}([-7.847359573362729, -1.3315947853190484, -0.5803510819209906, 1.1102764392598465, 9.988712842023517, 19.064552472027945, 22.360242601879634, 33.29037569667121, 44.30268624603188, 61.37029298584822  …  329.46633376403497, 367.3951090879816, 401.4586394961345, 429.0802630665355, 468.61496107560146, 507.32002150234865, 545.8993822489858, 589.8509862829196, 640.5342047989809, 675.5419963803967], [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5  …  10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0])

In [12]:
function tune_lengthscale(t, μ, N_e, N_c, M_adapt)
	N_e = max(1, N_e)

	if t <= M_adapt
		return 2μ * N_e / (N_e + N_c)
	else
		return μ
	end
end

function get_complementary(i, N)
	indices = collect(1:N)
	deleteat!(indices, i)
	return indices
end

function get_direction_vector(S, l, m, μ)
	return μ * (S[l].x - S[m].x)
end

""" 
	DifferentialMove(rng, k, μ, S, N)

	Perform a differential move for walker k.

	# Arguments
	- `rng::AbstractRNG`: Random number generator.
	- `k::Int`: Index of the walker.
	- `μ::Float64`: Lengthscale.
	- `S::Array{Float64, 2}`: Array of walker positions.
	- `N::Int`: Number of walkers.

	# Returns
	- `ηₖ::Array{Float64, 1}`: Differential move.
"""
function DifferentialMove(rng, k, μ, S, N)
	# work on walker k
	indices = get_complementary(k, N)
	# draw two random indices from the complementary set, without replacement
	l, m = sample(rng, indices, 2, replace = false)
	return get_direction_vector(S, l, m, μ)
end

DifferentialMove

In [13]:
using AbstractMCMC

struct EnsembleSliceSampler{T<:Float64,A<:Int64} <: AbstractMCMC.AbstractSampler
    "initial length scale"
    μ_init::T
    "number of adapation steps"
    M_adapt::A
    "number of walkers"
    N_walkers::A
    "max number of attempts"
    max_steps::A
end
struct Walker{A<:AbstractVector{<:Real}}
    "current position"
    x::A # a matrix of dimension n_params
end

struct ESState{A<:AbstractVector,T<:Float64,B<:Int64}
    # "current position"
    # vi::V
    "current position"
    x::A
    "length scale"
    μ::T
    "iteration"
    t::B
end

struct ESSample{A<:AbstractVector}
    "current position"
    x::A # a matrix of dimension n_walkers * n_params
end

In [89]:
function AbstractMCMC.step(
	rng::Random.AbstractRNG,
	model_wrapper::AbstractMCMC.LogDensityModel,
	sampler::EnsembleSliceSampler,
	state::ESState)

	model = model_wrapper.logdensity
	# extract the sampler parameters
	μ = sampler.μ_init
	M_adapt = sampler.M_adapt
	N_walkers = sampler.N_walkers
	max_steps = sampler.max_steps

	f(y) = LogDensityProblems.logdensity(model, y)

	# extract current state
	S, μ, t = state.x, state.μ, state.t
	N_dim = size(x, 2)

	x_new = Vector(undef, N_walkers)
	# Matrix{Float64}(undef, N_walkers, N_dim)

	R, L, N_e, N_c = 0, 0, 0, 0
	X′ = 0

	# loop over the walkers
	for k in 1:N_walkers
		Xₖ = S[k].x # get the current position of walker k
		ηₖ = DifferentialMove(rng, k, μ, S, N_walkers) # get the differential move

		δ = rand(rng, Exponential(1))
		Y = f(Xₖ) - δ

		L = -rand(rng)
		R = L + 1
		l = 0
		while Y < f(L .* ηₖ + Xₖ)
			L = L - 1
			N_e = N_e + 1
			l += 1
			if l == max_steps
				println("L: ", L, " Y: ", Y, " f(L): ", f(L .* ηₖ + Xₖ))
				error("Max steps reached", " iteration: ", t, " walker: ", k)
			end
		end
		l = 0
		while Y < f(R .* ηₖ + Xₖ)
			R = R + 1
			N_e = N_e + 1
			l += 1
			if l == max_steps
				println("L: ", R, " Y: ", Y, " f(R): ", f(R .* ηₖ + Xₖ))
				error("Max steps reached")
			end
		end

		l = 0
		while true
			l += 1
			X′ = rand(rng, Uniform(L, R))
			Y′ = f(X′ .* ηₖ + Xₖ)
			if Y < Y′
				break
			end
			if X′ < 0
				L = X′
				N_c = N_c + 1
			else
				R = X′
				N_c = N_c + 1
			end
			if l == max_steps
				println("L: ", R, " Y: ", Y, " f(R): ", f(R .* ηₖ + Xₖ))

				error("Max steps reached")
			end
		end

		Xₖ = X′ .* ηₖ + Xₖ
		x_new[k] = Walker(Xₖ)
		# push!(x_new, Walker(Xₖ))
	end
	# println("R: ", R, " L: ", L, " N_e: ", N_e, " N_c: ", N_c, " μ: ", μ)
	μ = tune_lengthscale(t, μ, N_e, N_c, M_adapt)
	t += 1
	# println((x_new) isa Vector{<:Walker})
	state_new = ESState(x_new, μ, t)
	return ESSample(x_new), state_new
end

In [80]:
rng = Random.default_rng(89)
ndims = 3
nwalkers = 2 * ndims
sampler = EnsembleSliceSampler(1.0, 100, 6, 10_000)
init_p = [ Walker(randn(rng,ndims)) for i in 1:nwalkers]
state = ESState(init_p, 1.0, 1)

ESState{Vector{Walker{Vector{Float64}}}, Float64, Int64}(Walker{Vector{Float64}}[Walker{Vector{Float64}}([-1.4700350877845583, -0.28079717563588386, 0.21819935075366134]), Walker{Vector{Float64}}([0.0015359654656915698, -1.292041156524825, -0.5001426097687957]), Walker{Vector{Float64}}([1.6207908277595857, 0.6888362954156618, -1.3282856773114313]), Walker{Vector{Float64}}([0.5969900613470707, 1.5204721141867714, -1.1102535821792976]), Walker{Vector{Float64}}([0.555143907575193, -2.7896723633694007, 0.9822429048406388]), Walker{Vector{Float64}}([1.2773868591229607, 1.6377902315278676, 0.09285335904755491])], 1.0, 1)

In [81]:
rng = Random.default_rng(132)

x_next, state_next = AbstractMCMC.step(
    rng,
    AbstractMCMC.LogDensityModel(MYmodel),
    sampler,
    state
)

(ESSample{Vector{Any}}(Any[Walker{Vector{Float64}}([-1.320428073770339, 0.6363172898812206, 0.033969218834172865]), Walker{Vector{Float64}}([3.7951771327792407, -0.10192513139156234, -2.398279039127977]), Walker{Vector{Float64}}([2.4451932864233825, 1.407245953793178, -1.8581195134353679]), Walker{Vector{Float64}}([1.982880798480703, 3.2158690377767565, -1.8190461466202503]), Walker{Vector{Float64}}([2.2543304577792065, -1.3089489421531624, -0.10980445470264522]), Walker{Vector{Float64}}([2.427923317230537, 7.072128628470728, -1.0860031172199034])]), ESState{Vector{Any}, Float64, Int64}(Any[Walker{Vector{Float64}}([-1.320428073770339, 0.6363172898812206, 0.033969218834172865]), Walker{Vector{Float64}}([3.7951771327792407, -0.10192513139156234, -2.398279039127977]), Walker{Vector{Float64}}([2.4451932864233825, 1.407245953793178, -1.8581195134353679]), Walker{Vector{Float64}}([1.982880798480703, 3.2158690377767565, -1.8190461466202503]), Walker{Vector{Float64}}([2.2543304577792065, -1.30

In [82]:
using DynamicPPL

In [83]:
function AbstractMCMC.step(
    rng::Random.AbstractRNG,
    model_wrapper::AbstractMCMC.LogDensityModel,
    ::EnsembleSliceSampler;
    kwargs...)
    model = model_wrapper.logdensity
    nwalkers = sampler.N_walkers
    ndims = LogDensityProblems.dimension(model)
    # x = randn(rng,nwalkers,ndims)
    init_p = [ Walker(randn(rng,ndims)) for i in 1:nwalkers]
    # x = [DynamicPPL.VarInfo(rng, model, DynamicPPL.SampleFromPrior()) for _ in 1:nwalkers]
    # x = mapreduce(permutedims, hcat, x)
    return ESSample(init_p),  ESState(init_p, 1.0, 1)
end

In [87]:

# samples = sample(MYmodel, sampler, 1_000; initial_state=state, progress=false)
# using MCMCChains
# samples_matrix = stack(sample -> sample.x, samples)
# samples_matrix = stack(sample -> sample.x, samples_matrix)
# samples_matrix = permutedims(samples_matrix, [3, 1, 2])

# ch = Chains(samples_matrix,[:α,:β,:γ])
# using StatsPlots
# plot(ch[100:end,:,:])

In [90]:
using LinearAlgebra

@model function InferenceModel(y, t)
	α ~ Normal(2, 1)
	β ~ Normal(0, 1)
	γ ~ Normal(-2, 1)
	return y ~ MvNormal(fun(t, α, β, γ), I)
end

mymod = InferenceModel(y, t)
# Turing.Inference.getparams(::Turing.Model, sample::ESSample) = sample.x
function Turing.Inference.getparams(::Turing.Model, sample::ESSample) 
	# for walker in sample.x
	# 	println(walker.x)
	# end
	return [sample.x[i].x for i in 1:length(sample.x)]
end

In [91]:
vi = DynamicPPL.VarInfo(rng, mymod, DynamicPPL.SampleFromPrior()) 

TypedVarInfo{@NamedTuple{α::DynamicPPL.Metadata{Dict{VarName{:α, typeof(identity)}, Int64}, Vector{Normal{Float64}}, Vector{VarName{:α, typeof(identity)}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, β::DynamicPPL.Metadata{Dict{VarName{:β, typeof(identity)}, Int64}, Vector{Normal{Float64}}, Vector{VarName{:β, typeof(identity)}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, γ::DynamicPPL.Metadata{Dict{VarName{:γ, typeof(identity)}, Int64}, Vector{Normal{Float64}}, Vector{VarName{:γ, typeof(identity)}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}}, Float64}((α = DynamicPPL.Metadata{Dict{VarName{:α, typeof(identity)}, Int64}, Vector{Normal{Float64}}, Vector{VarName{:α, typeof(identity)}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}(Dict(α => 1), [α], UnitRange{Int64}[1:1], [0.4993117530487674], Normal{Float64}[Normal{Float64}(μ=2.0, σ=1.0)], Set{DynamicPPL.Selector}[Set()], [0], Dict{String, BitVector}("del" => [0], "trans" => [0])), β = DynamicPPL.Metadata{Di

In [93]:
Turing.Inference.Transition

Turing.Inference.Transition

In [73]:
# sampler = Emcee(50)

In [74]:
# chain = sample(mymod, sampler, 10_00)

In [92]:
chain = sample(mymod, externalsampler(sampler), 100)

Sampling   0%|                                          |  ETA: N/A
Sampling 100%|██████████████████████████████████████████| Time: 0:00:00


MethodError: MethodError: no method matching DynamicPPL.Metadata(::Dict{VarName{:α, typeof(identity)}, Int64}, ::Vector{VarName{:α, typeof(identity)}}, ::Vector{UnitRange{Int64}}, ::Vector{Vector{Float64}}, ::Vector{Normal{Float64}}, ::Vector{Set{DynamicPPL.Selector}}, ::Vector{Int64}, ::Dict{String, BitVector})

Closest candidates are:
  DynamicPPL.Metadata(::TIdcs, ::TVN, ::Vector{UnitRange{Int64}}, !Matched::TVal, ::TDists, ::TGIds, ::Vector{Int64}, ::Dict{String, BitVector}) where {TIdcs<:(Dict{<:VarName, Int64}), TDists<:(AbstractVector{<:Distribution}), TVN<:(AbstractVector{<:VarName}), TVal<:(AbstractVector{<:Real}), TGIds<:AbstractVector{Set{DynamicPPL.Selector}}}
   @ DynamicPPL ~/.julia/packages/DynamicPPL/DvdZw/src/varinfo.jl:47


In [ ]:
samples_matrix = stack(sample -> sample.x, samples)
ch = Chains(permutedims(samples_matrix, [3, 2, 1]),[:α,:β,:γ])

In [ ]:
ch[100:end,:,:]

In [ ]:
using StatsPlots
plot(ch[100:end,:,:])

In [131]:
# using DelimitedFiles
# data = readdlm("data.txt")
# t, y, yerr = data[:, 1], data[:, 2], data[:, 3]


# Turing.Inference.getparams(::Turing.Model, sample::ESSample) = sample.x

# fun(x,α,β,γ) = @. α * x^2 + β * x + γ
# x = 0.0:0.5:15.0

# @model function InferenceModel(y)
# 	α ~ Normal(2, 1)
# 	β ~ Normal(0, 1)
# 	γ ~ Normal(-2, 1)

# 	y ~ MvNormal(fun(x, α, β, γ), 1)
# end

# m = InferenceModel(y)
# m2 = m | (α=1.0, β=0.3, γ=-2.)

In [132]:
# DynamicPPL.SampleFromPrior()